In this notebook, we simulate the field created by the TRIF electromagnet and save it as a data set. This data set is going to be used to test our calibration approach.

The electromagnet has three poles, that we model as three magnetic dipoles oriented in the same direction (physically, each pole is a coil wrapped around a ferromagnetic core). The dipoles are located in the same plain at the same distance from the axis of symmetry, and their position vectors are at $120^0$ with respect to each other.

Here we create a set of various dipole magnitudes and calculate the corresponding magnetic field.

In the experiment each dipole magnitude corresponds to a voltage applied to the corresponding coil inside the magnet. In case of linear relationship between the voltage and magnetic field vector, it is enough to collect and combine the field-vs-voltage characteristic from each individual coil. We'll save this data set separately as linear_calibration_set.csv.

In the non-linear case, in order to properly train an artificial neural network, we'll additionally need data with all coils receiving voltage simultaneously.

Since a constant ambient field can be easily dealt with by subtracting its value from the data set, we do not consider it here.

***

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Functions for the field calculations

def mag(x):
    """Calculates magnitude of vector x"""
    return np.sqrt(np.dot(x,x))

def dipole_field(r, r0, m, a):
    """
    Calculates magnetic field of a single dipole
    r - location of the field
    r0 - position of the dipole
    m - magnitude of the dipole. Negative values correspond to the opposite orientation of  the dipole axis
    a - dipole axis"""
    M = m*a   # Magnetic moment vector
    R = r-r0
    if np.dot(R,R) == 0: #This one is to avoid singularities
        return np.array([0, 0, 0])
    return mu0*(3*R*(np.dot(M,R))/(mag(R)**5) - M/(mag(R)**3))

def total_field(r, m_array: np.array, dipole_axis: np.array) -> np.array:
    """Calculates the total field at point r
    To simplify the data set generation later, the magnitudes of the three dipoles are placed in array m.
    B0 is the external field.
    dipole_axis is the dipole orientation (same for all three)"""
    B1 = dipole_field(r, R1, m_array[0], dipole_axis)
    B2 = dipole_field(r, R2, m_array[1], dipole_axis)
    B3 = dipole_field(r, R3, m_array[2], dipole_axis)
    return B1+B2+B3

In [3]:
# Constants and the setup parameters

mu0 = 10**-7# magnetic constant
m = 1e8 # Scaling factor magnetic dipole magnitude

# Magnetic dipole position vectors
R1 = np.array([1, 0, 1])
R2 = np.array([-np.cos(np.pi/3), np.sin(np.pi/3), 1])
R3 = np.array([-np.cos(np.pi/3), -np.sin(np.pi/3), 1])

# Magnetic dipole orientation (same for all three)
A = np.array([0, 0, 1]) # It is a global variable

# The magnetometer location. Magnetic field is calculated here
r0 = np.array([0., 0., 0.])

# Voltage range
v_range = 10

# Non-linearity factor
nf = 15

***

# Linear case

In [4]:
# Apply voltage to a single coil in a linear fashion

N1 = 512 # Number of training samples for each dipole

V1 = np.concatenate((np.linspace(-v_range, v_range, N1), np.zeros(N1), np.zeros(N1)))
V2 = np.concatenate((np.zeros(N1), np.linspace(-v_range, v_range, N1), np.zeros(N1)))
V3 = np.concatenate((np.zeros(N1), np.zeros(N1), np.linspace(-v_range, v_range, N1)))

V_ind = np.stack((V1, V2, V3), axis=1) # Stack vertically

In [5]:
# Calculate magnetic field

B_ind = []
for V in V_ind:
    B_ind.append(total_field(r0, V * m, A))

individual_calibration_set = pd.DataFrame(np.concatenate((B_ind, V_ind), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"]) # Combine fields and voltages into a single dataset

In [6]:
individual_calibration_set.head()

,B_x,B_y,B_z,V_1,V_2,V_3
0,-53.033009,0.0,-17.677670,-10.000000,0.0,0.0
1,-52.825443,0.0,-17.608481,-9.960861,0.0,0.0
2,-52.617877,0.0,-17.539292,-9.921722,0.0,0.0
3,-52.410312,0.0,-17.470104,-9.882583,0.0,0.0
4,-52.202746,0.0,-17.400915,-9.843444,0.0,0.0


In [7]:
# Save
individual_calibration_set.to_csv('data/individual_calibration_set.csv')

***

# Non-linear case
As the non-linearity, we apply a hyperbolic tangent to the voltage before calculating the field. Since in this case we'll need to use deep learning, we generate two sets: one for training and one for validation.


## Training set

For visualization purposes, to compare to the linear case, we create a separate set with only one magnetic pole activated at a time, by applying the non-linearity to the voltage from linear set.

In [8]:
B_nl_ind = []
for V in V_ind:
    B_nl_ind.append(total_field(r0, v_range*np.tanh(V/nf) * m, A))

nl_individual_calibration_set = pd.DataFrame(np.concatenate((B_nl_ind, V_ind), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"]) # Combine fields and voltages into a single dataset

In [9]:
# Save
nl_individual_calibration_set.to_csv('data/nl_individual_calibration_set.csv')

In [10]:

# Apply random voltages to all three coils
N2 = 512 # Total number of values
V_rand = np.random.uniform(-v_range, v_range, (N2, 3)) # Random voltages between -10 and 10

# Calculate magnetic field

B_rand = []
for V in V_rand:
    B_rand.append(total_field(r0, v_range*np.tanh(V/nf) * m, A))

# stack with the individual voltage set
V_full = np.concatenate((V_ind, V_rand))
B_full = np.concatenate((B_nl_ind, B_rand))

full_calibration_set = pd.DataFrame(np.concatenate((B_full, V_full), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"]) # Combine fields and voltages into a single dataset
print(full_calibration_set.head())
print(full_calibration_set.tail())

         B_x  B_y        B_z        V_1  V_2  V_3
0 -30.906733  0.0 -10.302244 -10.000000  0.0  0.0
1 -30.815215  0.0 -10.271738  -9.960861  0.0  0.0
2 -30.723419  0.0 -10.241140  -9.921722  0.0  0.0
3 -30.631345  0.0 -10.210448  -9.882583  0.0  0.0
4 -30.538993  0.0 -10.179664  -9.843444  0.0  0.0
            B_x        B_y        B_z       V_1       V_2       V_3
2043  -6.361425  40.639027  -4.620541 -2.530133  6.270636 -8.032289
2044  36.827881 -27.012424   0.871548  7.832870 -8.423070  1.187933
2045 -18.899898   5.447426  -0.508392 -3.785993  2.552007  0.749166
2046 -51.192099  -0.026049   4.954242 -9.278019  6.622681  6.632960
2047   4.801533  -4.879224  14.438066  5.186331  2.868706  4.563560


In [11]:
# Save
full_calibration_set.to_csv('data/full_calibration_set.csv')

### Validation set for deep learning

We'll use this set to monitor the neural network performance during training in the non-linear case.

In [12]:
N_val = 256

In [13]:

V_val = np.random.uniform(-v_range, v_range, (N_val, 3)) # Random voltages between -10 and 10
B_val = []

for V in V_val:
    B_val.append(total_field(r0, v_range*np.tanh(V/nf) * m, A))

val_set = pd.DataFrame(np.concatenate((B_val, V_val), axis = 1), columns = ["B_x", "B_y", "B_z", "V_1", "V_2", "V_3"])

In [14]:
val_set.head()

,B_x,B_y,B_z,V_1,V_2,V_3
0,-35.054605,-31.478704,-4.256388,-8.664089,-3.082010,7.898837
1,-3.670747,0.012866,6.129149,1.043101,2.095290,2.091005
2,-17.755327,-23.690236,-18.060738,-9.574809,-7.979126,0.434361
3,3.832458,38.245194,7.151156,2.776592,8.792145,-4.735372
4,-1.010546,29.079711,3.967122,0.932721,6.314353,-3.598850


In [15]:
# Save
val_set.to_csv('data/validation_set.csv')